In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
#from nltk.stem import SnowballStemmer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC,LinearSVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score
from sklearn.preprocessing import label_binarize

#from gensim.models import Word2Vec
from gensim.test.utils import common_texts
from gensim.sklearn_api import W2VTransformer
from gensim.models.wrappers import FastText


In [71]:
data_train = pd.read_table("all/train.tsv")
#data_test = pd.read_table("all/test.tsv") # No sentiment labels, splitting training set into train/test : 66/33

# data has parsed sentences as separate entries, let's just try analyzing the full reviews first
data_train_drop   = data_train#.drop_duplicates(subset=['SentenceId'],keep='first')
#data_test_drop    = data_test.drop_duplicates(subset=['SentenceId'],keep='first')

data_train_x, data_test_x, data_train_y, data_test_y = train_test_split(data_train_drop['Phrase'], data_train_drop['Sentiment'], test_size=0.33)


In [78]:
# from here on, test refers to the test set from the "training" tsv
en_stopwords  = set(stopwords.words("english"))

#sci kit learn: BoW
#vectorizer = CountVectorizer(analyzer='word',stop_words=en_stopwords)
#gensim: Word2Vec
vectorizer = W2VTransformer(size=1000, min_count=1, seed=1)#Word2Vec(data_train_x)
vectorizer.fit(data_train['Phrase'])
#df_train_x = vectorizer.fit_transform(data_train_x)
df_train_x = vectorizer.transform(data_train_x)
df_test_x  = vectorizer.transform(data_test_x)
df_train_y = data_train_y#label_binarize(data_train_y, classes=[0, 1, 2, 3, 4])
df_test_y  = data_test_y#label_binarize(data_test_y, classes=[0, 1, 2, 3, 4])


KeyError: "word 'ironic exception' not in vocabulary"

In [ ]:
kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [ ]:
# model(s?)
#svm_model = SVC(probability=True, kernel="rbf", class_weight="balanced",
#                decision_function_shape="ovr",gamma="scale",max_iter=1000,tol=1e-05)
svm_model = LinearSVC(
    C=1.0,
    class_weight='balanced',
    #dual=True,
    #fit_intercept=True,
    #intercept_scaling=1,
    #loss='squared_hinge',
    max_iter=10000,
    multi_class='ovr',
    #penalty='l2',
    #random_state=0,
    #tol=1e-05, 
    verbose=0
)
#print(svm_model.get_params().keys())
pipeline_svm = make_pipeline(vectorizer, svm_model)

grid_svm = GridSearchCV(pipeline_svm,
                   param_grid = {'linearsvc__C': [0.01, 0.1, 1],'countvectorizer__ngram_range' : [(1,1),(1,2),(1,3)]}, 
                   cv = kfolds,
                   scoring="accuracy",
                   verbose=1,   
                   n_jobs=-1)
#svm_model.fit(df_train_x,df_train_y)
grid_svm.fit(data_train_x,df_train_y)


In [ ]:
#svm_model.score(df_test_x,df_test_y)
grid_svm.score(data_test_x,df_test_y)

In [30]:
svm_model.score(df_test_x,df_test_y) # big oof performance

0.29626998223801065

In [35]:
dec = svm_model.decision_function(df_test_x[0])
dec.shape[1]

10